In [410]:
class Data:
    
    def __init__(self,train_path,test_path,cible_path,header_path,sep): 
        
        import pandas as pd
    
        self.train_path=train_path
        self.test_path=test_path
        self.cible_path=cible_path
        self.header_path=header_path
        self.sep=sep
        self.train=pd.read_table(self.train_path, sep=self.sep,header=None)
        self.test=pd.read_table(self.test_path, sep=self.sep,header=None)
        self.cible=pd.read_table(self.cible_path,sep=self.sep,header=None)
        self.header=pd.read_csv(self.header_path,sep=self.sep,header=None)
               
    @property
    def test_path(self):
        return self._test_path

    @test_path.setter
    def test_path(self, v):
        self._test_path  =  v 
        
    @property
    def train_path(self):
        return self._train_path

    @train_path.setter
    def train_path(self, v):
        self._train_path  =  v 
        
    @property
    def train(self):
        return self._train

    @train.setter
    def train(self, v):
        self._train  =  v
        
    @property
    def alldata(self):
        return self._alldata

    @alldata.setter
    def alldata(self, v):
        self._alldata  =  v 
        
    @property
    def header(self):
        return self._header

    @header.setter
    def header(self, v):
        self._header  =  v
        
    @property
    def cible(self):
        return self._cible

    @cible.setter
    def cible(self, v):
        self._cible  =  v 
                
    @property
    def sep(self):
        return self._sep

    @sep.setter
    def sep(self, v):
        self._sep  =  v 
        
    @property
    def feature_name(self):
        return self._feature_name

    @feature_name.setter
    def feature_name(self, v):
        self._feature_name  =  v 
    
    # Create a the list of features name based on the header
    def createFeaturesName(self,indexToRemove,colToRemove,name='xForm'):
        self.feature_name=self.header.drop(self.header.columns[colToRemove], axis=1)
        if name=='xForm':    
            self.feature_name=self.feature_name.drop(self.feature_name.index[indexToRemove])[1]
        else:
            self.feature_name=self.feature_name.drop(self.feature_name.index[indexToRemove])[3]

    # Put headers to the data      
    def nameData(self):
        self.train.columns=self.feature_name
        self.test.columns=self.feature_name
        
        return self
        
    # Put Train/Test together     
    def joinData(self):
        
        import pandas as pd

        frames=[self.train,self.test]
        self.alldata= pd.concat(frames,keys=['train','test'])
        
        return self

    # Sous fonction à généralier..
    def oneHot(self,on='all'):
        if on=='all':
            self.alldata['x51']=self.alldata['x51']-2011
        
        return self
    
    
    def takeTrain(self):
    
        return self.alldata.ix['train']
 
 
    def takeTest(self):
    
        return self.alldata.ix['test']

In [411]:
from sklearn.base import BaseEstimator,TransformerMixin

In [412]:
class CreateAllData(TransformerMixin):
    
    """ Add the test to train """

    def __init__(self,testSet):
        self.testSet=testSet
        
    def fit(self,X,y=None):
        return self
    
    def transform(self,X):
        
        import pandas as pd

        frames=[X,self.testSet]
        aldata= pd.concat(frames,keys=['train','test'])
             
        return alldata
    

In [413]:
class Drop(TransformerMixin):
    
    """ Scale data"""

    def __init__(self,colsToDrop):
        self.colsToDrop=colsToDrop
        
    def fit(self,X,y=None):
        return self
    
    def transform(self,X):
        
        alldata_scale=X.drop(X[self.colsToDrop], axis=1)
             
        return alldata_scale
    

In [414]:
class Scale(TransformerMixin):
    
    """ Scale data"""

    def fit(self,X,y=None):
        return self
    
    def transform(self,X):
        
        alldata_scale=((X-X.mean())/(X.var()))
        
        return alldata_scale
    

In [415]:
class FillNaN_KNN(TransformerMixin):
    
    """ Fill Nan is data using KNN implementation of fancyimpute"""
    
    def __init__(self,k):
        self.numberCluster=k

    def fit(self,X,y=None):
        return self
    
    def transform(self,X):
        from fancyimpute import KNN
        
        alldata_filled = KNN(k=self.numberCluster,verbose=False).complete(X)
                
        return alldata_filled
    

In [416]:
class RebuildDF(TransformerMixin):
    
    """Rebuild from an ndarray a Dataframe with the original index"""
    
    def __init__(self,dataInit,colsToAdd):
        self.dataInit=dataInit
        self.colsToAdd=colsToAdd

    def fit(self,X,y=None):
        return self
    
    def transform(self,X):
        import pandas as pd
        
        datacomplete=pd.DataFrame(X)
        datacomplete.index=self.dataInit.index
        datacomplete[self.colsToAdd]=self.dataInit[self.colsToAdd]
        datacomplete.columns=self.dataInit.columns

        
        return datacomplete
    

In [417]:
class Take_train(TransformerMixin):
    
    """Rebuild from an ndarray a Dataframe with the original index"""
    
    def fit(self,X,y=None):
        return self
    
    def transform(self,X):
        import numpy as np
        train=X.ix['train']
                
        return train

In [379]:
data=Data(train_path="../Data_competition/X_train.txt"
                      ,test_path="../Data_competition/X_test.txt"
                      ,cible_path="../Data_competition/y_train.txt"
                      ,header_path="../Data_competition/header.csv"
                      ,sep=',')

data.createFeaturesName(colToRemove=[2],indexToRemove=[56,49,30,20,14,0,38],name='xForm')
data.nameData().joinData().oneHot()

In [370]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.base import BaseEstimator

In [314]:
pipeline=Pipeline([('drop',Drop(['x51']))
                   ,('scale',Scale())
                   ,('knn', FillNaN_KNN(k=3))
                   ,('rebuildDF',RebuildDF(data.alldata,['x51']))
                   ,('takeTrain',Take_train())
                   ,('gb',GradientBoostingClassifier())
                   ])

In [337]:
from sklearn.model_selection import GridSearchCV

La grid search sur le pipeline entier ne va pas fonctionner. En effet, gridsearch regarde au départ si ses deux entrées sont compatible. Ici comme on donne en entrée tout le dataset c'est plus gros que la cible. On va pouvoir ajouter deux classes qui vont construire le dataset à l'intérieur du transform. Voir partie 2.

In [418]:
data=Data(train_path="../Data_competition/X_train.txt"
                      ,test_path="../Data_competition/X_test.txt"
                      ,cible_path="../Data_competition/y_train.txt"
                      ,header_path="../Data_competition/header.csv"
                      ,sep=',')

data.createFeaturesName(colToRemove=[2],indexToRemove=[56,49,30,20,14,0,38],name='xForm')
data.nameData().joinData().oneHot()

In [407]:
pipeline=Pipeline([('drop',Drop(['x51']))
                   ,('scale',Scale())
                   ,('knn', FillNaN_KNN(k=5))
                   ,('rebuildDF',RebuildDF(data.alldata,['x51']))
                   ,('takeTrain',Take_train())
        ])

In [408]:
X=pipeline.transform(data.alldata)

In [409]:
tuned_parameters = [{'loss': ['deviance','exponential'],
                    'max_features':[5,10,20,50],
                    'max_depth':[3,5,10,30],
                    'n_estimators':[100,300]}]


clf = GridSearchCV(GradientBoostingClassifier(), tuned_parameters, cv=10,verbose=1)
clf.fit(X,data.cible[0].values)


print("Best parameters set found on development set:")
print()
print(clf.best_params_)
print()

print("Grid scores on development set:")
print()
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
            % (mean, std * 2, params))
print()

Fitting 10 folds for each of 64 candidates, totalling 640 fits


KeyboardInterrupt: 

In [344]:
clf.best_score_

0.95541666666666669

## Contourner la difficulté de GridSearch

In [389]:
data=Data(train_path="../Data_competition/X_train.txt"
                      ,test_path="../Data_competition/X_test.txt"
                      ,cible_path="../Data_competition/y_train.txt"
                      ,header_path="../Data_competition/header.csv"
                      ,sep=',')

data.createFeaturesName(colToRemove=[2],indexToRemove=[56,49,30,20,14,0,38],name='xForm')
data.nameData().joinData().oneHot()

In [390]:
pipeline=Pipeline([('addTest',CreateAllData(data.takeTest()))
                   ,('drop',Drop(['x51']))
                   ,('scale',Scale())
                   ,('knn', FillNaN_KNN(k=3))
                   ,('rebuildDF',RebuildDF(data.alldata,['x51']))
                   ,('gbt',GradientBoostingClassifier())
                    ])

In [391]:
tuned_parameters = [{'loss': ['deviance','exponential'],
                    'max_features':[5,10,20,50],
                    'max_depth':[3,5,10,30],
                    'n_estimators':[100,300]}]


clf = GridSearchCV(pipeline, tuned_parameters, cv=10,verbose=1)
clf.fit(data.takeTrain(),data.cible[0].values)


print("Best parameters set found on development set:")
print()
print(clf.best_params_)
print()

print("Grid scores on development set:")
print()
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
            % (mean, std * 2, params))
print()

Fitting 10 folds for each of 64 candidates, totalling 640 fits


ValueError: Invalid parameter max_features for estimator Pipeline. Check the list of available parameters with `estimator.get_params().keys()`.

## Fit juste avec le train

In [365]:
pipeline=Pipeline([('drop',Drop(['x51']))
                   ,('scale',Scale())
                   ,('knn', FillNaN_KNN(k=7))
                   ,('rebuildDF',RebuildDF(data.alldata,['x51']))
                   ,('gbt',GradientBoostingClassifier())
                    ])

In [366]:
tuned_parameters = [{'loss': ['deviance','exponential'],
                    'max_features':[5,10,20,50],
                    'max_depth':[3,5,10,30],
                    'n_estimators':[100,300]}]


clf = GridSearchCV(pipeline, tuned_parameters, cv=10,verbose=1)
clf.fit(data.takeTrain(),data.cible[0].values)


print("Best parameters set found on development set:")
print()
print(clf.best_params_)
print()

print("Grid scores on development set:")
print()
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
            % (mean, std * 2, params))
print()

Fitting 10 folds for each of 64 candidates, totalling 640 fits


AttributeError: 'Drop' object has no attribute 'get_params'